


# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint
### PyTorch Custom Dataloader

#### Setup Steps

In [0]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "P19A06E_test" #@param {type:"string"}


In [0]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "981234567" #@param {type:"string"}


In [0]:
#@title Run this cell to complete the setup for this Notebook

from IPython import get_ipython
ipython = get_ipython()
  
notebook="BLR_M2W7_DEMO_EXP4" #name of the notebook

def setup():
    ipython.magic("sx pip3 install torch")
    ipython.magic("sx pip3 install torchvision")
    ipython.magic("sx pip3 install librosa")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/week7/Speech_samples.zip")
    ipython.magic("sx unzip Speech_samples.zip")
    print ("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "id" : Id, "file_hash" : file_hash, "notebook" : notebook}

      r = requests.post(url, data = data)
      print("Your submission is successful. Ref:", submission_id)
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if Additional: return Additional      
    else: raise NameError('')
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
  
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


In [0]:
import torch 
import torchvision
import torch.nn as nn
#import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import librosa
import os

In [0]:
!ls Speech_samples/

0_a1_21.wav   0_b2_93.wav   1_b4_96.wav   1_b7_48.wav	2_b2_37.wav
0_a1_33.wav   0_b2_9.wav    1_b5_0.wav	  1_b7_60.wav	2_b2_49.wav
0_a1_57.wav   0_b3_21.wav   1_b5_108.wav  2_a1_13.wav	2_b2_61.wav
0_a1_69.wav   0_b3_33.wav   1_b5_12.wav   2_a1_1.wav	2_b2_73.wav
0_a1_81.wav   0_b3_45.wav   1_b5_24.wav   2_a1_25.wav	2_b2_85.wav
0_a1_93.wav   0_b3_57.wav   1_b5_36.wav   2_a1_37.wav	2_b2_97.wav
0_a1_9.wav    0_b3_69.wav   1_b5_48.wav   2_a1_49.wav	2_b3_109.wav
0_a2_0.wav    0_b3_9.wav    1_b5_60.wav   2_a1_61.wav	2_b3_13.wav
0_a2_12.wav   1_b3_108.wav  1_b5_72.wav   2_a1_73.wav	2_b3_1.wav
0_a2_24.wav   1_b3_24.wav   1_b5_84.wav   2_a1_85.wav	2_b3_25.wav
0_a3_0.wav    1_b3_36.wav   1_b5_96.wav   2_a1_97.wav	2_b3_37.wav
0_a3_12.wav   1_b3_48.wav   1_b6_0.wav	  2_a2_14.wav	2_b3_49.wav
0_a3_24.wav   1_b3_60.wav   1_b6_108.wav  2_a2_26.wav	2_b3_61.wav
0_a3_36.wav   1_b3_72.wav   1_b6_12.wav   2_a2_2.wav	2_b3_73.wav
0_a3_48.wav   1_b3_84.wav   1_b6_24.wav   2_a3_14.wav	2_b3_85.wav
0_a4_0.wav    

###Custom dataset should inherit torch.utils.data.Dataset and implement __init_(), __len__() and __getitem__() functions
###For more information, explore the data loading tutorial given at:
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [0]:
class CustomDataset(torch.utils.data.Dataset):    ### custom data loader
    
    def __init__(self, audio_files_root_dir, audio_file_list, n_mfcc=30, n_mels=128, frames = 10):
        self.audio_root = audio_files_root_dir
        self.audio_file_list = audio_file_list
        self.n_mfcc = n_mfcc
        self.n_mels = n_mels
        self.frames = frames
        
    def __len__(self):
        return len(self.audio_file_list)

    def __getitem__(self, index):
        
        try :
            #don't worry with the below code; typically here you write the way you read your data (e.g. speech, vision, text).
            
            audio_file = os.path.join(self.audio_root, self.audio_file_list[index])
            y, sr = librosa.load(audio_file, sr=8000)
            D = np.abs(librosa.stft(y))**2
            #S = librosa.feature.melspectrogram(S=D)
            S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=self.n_mels,fmax=8000)
            log_S = librosa.power_to_db(S,ref=np.max)
            features = librosa.feature.mfcc(S=log_S, n_mfcc=self.n_mfcc)
            if features.shape[1] < self.frames :
                features = np.hstack((features, np.zeros((self.n_mfcc, self.frames - features.shape[1]))))
            elif features.shape[1] > self.frames:
                features = features[:, :self.frames]
            label = int(self.audio_file_list[index].split("_")[0])
        except :
            print(audio_file)

        return features.flatten(), label

In [0]:
audio_root = "Speech_samples"

In [0]:
audio_file_list = os.listdir(audio_root)
print(len(audio_file_list))
audio_file_list = [i for i in audio_file_list if i.find(".wav") > 0]
print(len(audio_file_list))

121
121


In [0]:
train_ratio = 0.9

In [0]:
train_files_list = audio_file_list[: int(len(audio_file_list)*train_ratio)]
test_files_list = audio_file_list[int(len(audio_file_list)*train_ratio):]

In [0]:
print(len(train_files_list), len(test_files_list))

108 13


In [0]:
# You can then use the prebuilt data loader. 
train_set = CustomDataset("Speech_samples", train_files_list)
test_set = CustomDataset("Speech_samples", test_files_list)

In [0]:
#Dataloader is used to load data in mini batches. Here shuffle param refers to whether you want to sample the data randomly
trainloader = torch.utils.data.DataLoader(train_set,batch_size=10, shuffle = True)

testloader = torch.utils.data.DataLoader(test_set,batch_size=1, shuffle = False)

In [0]:
#Iterating over the train data laoder
for samples, labels  in trainloader:
    print(samples.size(), labels.size())
    break

/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


torch.Size([10, 300]) torch.Size([10])


In [0]:
#Iterating over the test data laoder
for samples, labels  in testloader:
    print(samples.size(), labels.size())
    break

torch.Size([1, 300]) torch.Size([1])


/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


### Please answer the questions below to complete the experiment:

In [0]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Too Simple, I am wasting time" #@param ["Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging me", "Was Tough, but I did it", "Too Difficult for me"]


In [0]:
#@title If it was very easy, what more you would have liked to have been added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "test" #@param {type:"string"}

In [0]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["Yes", "No"]

In [0]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful. Ref: 9453
